# import files and directories

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import uuid
from PIL import Image
import torch
import torchvision.transforms.functional as F
import random
from torch.utils.data import ConcatDataset, DataLoader, SubsetRandomSampler
from torchvision import transforms
from torch import nn
from torch.utils.data import Dataset
from torch import flatten
import torch.nn.functional as Fnn
from torch.optim.lr_scheduler import StepLR

In [2]:
POS_PATH = os.path.join("..",'data','positive')
NEG_PATH = os.path.join("..",'data','negetive')
ANC_PATH = os.path.join("..",'data','anchor')

In [4]:
# add directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

# Untar Labelled Faces in the Wild Dataset

In [7]:
!tar -xf lfw.tgz

In [8]:
for directory in os.listdir('lfw'):
  for file in os.listdir(os.path.join('lfw',directory)):
    EX_PATH = os.path.join('lfw',directory, file)
    NEW_PATH = os.path.join(NEG_PATH, file)
    os.replace(EX_PATH, NEW_PATH)

In [3]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

# Data Augmentation

In [59]:
import torchvision.transforms.functional as F
import random
def data_aug(img):
    flag = [0,0,0,0]

    data = []
    while not all(x == 1 for x in flag):
        if flag[0] == 0 and random.randint(0,100) / 100 < 0.5:
            img = F.adjust_brightness(img, brightness_factor=1.1)
            data.append(img)
            flag[0] = 1
        
        if flag[1] == 0 and random.randint(0,100) / 100 < 0.5:
            img = F.adjust_contrast(img, contrast_factor=torch.empty(1).uniform_(0.6, 1).item())
            data.append(img)
            flag[1] = 1
            
        if flag[2] == 0 and random.randint(0,100) / 100 < 0.5:
            img = F.hflip(img)
            data.append(img)
            flag[2] = 1
            
        if flag[3] == 0 and random.randint(0,100) / 100 < 0.5:
            img = F.adjust_saturation(img, saturation_factor=torch.empty(1).uniform_(0.9, 1).item())
            data.append(img)
            flag[3] = 1
        
        
    return data


In [ ]:
for file_name in os.listdir(os.path.join(POS_PATH)):
    img_path = os.path.join(POS_PATH, file_name)
    img = Image.open(img_path).convert('RGB')
    img_tensor = F.to_tensor(img)
    augmented_images = data_aug(img_tensor)
    for i, image in enumerate(augmented_images):
        image = F.to_pil_image(image)
        image.save(os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1())))

# Load Data in Dataloader

In [3]:
class MergeImageDataset(Dataset):
    def __init__(self,ANC_PATH  ,POS_PATH, NEG_PATH, types ,transform = None):
        
        self.POS_PATH = POS_PATH
        self.NEG_PATH = NEG_PATH
        self.ANC_PATH = ANC_PATH
        
        self.types = types
        
        self.transform = transform
        
        self.POS_IMG = os.listdir(POS_PATH)
        self.NEG_IMG = os.listdir(NEG_PATH)
        self.ANC_IMG = os.listdir(ANC_PATH)
        
    def __len__(self):
        return len(self.ANC_IMG)

    def __getitem__(self, idx):
        
        anc_dir  = os.path.join(self.ANC_PATH, self.ANC_IMG[idx])
        anc_image = Image.open(anc_dir).convert('RGB')
        
        if self.types == 1: 
            pos_dir  = os.path.join(self.POS_PATH, self.POS_IMG[idx])
            pos_image = Image.open(pos_dir).convert('RGB')

        if self.types == 0: 
            neg_dir  = os.path.join(self.NEG_PATH, self.NEG_IMG[idx])
            neg_image = Image.open(neg_dir).convert('RGB')
        
        data = [anc_image, pos_image if self.types == 1 else neg_image, torch.ones(1) if self.types == 1 else torch.zeros(1) ]

        if self.transform:
            data[0] = self.transform(data[0])
            data[1] = self.transform(data[1])
        
        return data

In [4]:
transform = transforms.Compose([transforms.Resize(100),
                                transforms.ToTensor()])

In [5]:
positive = MergeImageDataset(ANC_PATH, POS_PATH, NEG_PATH, types = 1, transform = transform)
negtive = MergeImageDataset(ANC_PATH, POS_PATH, NEG_PATH, types = 0, transform = transform)

In [6]:
data = ConcatDataset([negtive,positive])

In [7]:
# Split the individual datasets into training and validation parts
num_samples = len(data)
indices = list(range(num_samples))
np.random.shuffle(indices)
split = int(np.floor(0.2 * num_samples))  # Use 20% of data for validation
train_indices, val_indices = indices[split:], indices[:split]

# Create samplers for the training and validation parts
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)


In [8]:
# Create data loaders for the training and validation parts
train_loader = DataLoader(data, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(data, batch_size=32, sampler=val_sampler)

# Build Model

In [15]:
class Embedding(nn.Module):
    def __init__(self):
            super(Embedding, self).__init__()

            self.conv_1 = nn.Conv2d(3, 64,kernel_size=(10, 10))
            self.relu_1 = nn.ReLU()
            self.maxpool_1 = nn.MaxPool2d(2, stride = 2 ,padding = 1)
            
            self.conv_2 =  nn.Conv2d(64, 128,kernel_size=(7, 7))
            self.relu_2 =  nn.ReLU()
            self.maxpool_2 = nn.MaxPool2d(2 , 2)
            
            self.conv_3 =  nn.Conv2d(128, 128,kernel_size=(4, 4))
            self.relu_3 =  nn.ReLU()
            self.maxpool_3 =  nn.MaxPool2d(2, 2, padding = 1)
            
            self.conv_4 =  nn.Conv2d(128, 256,kernel_size=(4, 4))
            self.relu_4 =  nn.ReLU()
            self.flatten = nn.Flatten(start_dim=1)
            
            self.linear = nn.Linear(256 * 6 * 6, 5000)
        
    def forward(self, x):
        x = self.conv_1(x)
        x = self.relu_1(x)
        x = self.maxpool_1(x)
        
        x = self.conv_2(x)
        x = self.relu_2(x)
        x = self.maxpool_2(x)
        
        x = self.conv_3(x)
        x = self.relu_3(x)
        x = self.maxpool_3(x)
        
        x = self.conv_4(x)
        x = self.relu_4(x)
        
        x = self.flatten(x)
        
        x = self.linear(x)
        
        return x
        

In [16]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        
        self.embeder = Embedding()
        self.classifier = nn.Linear(5000,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, input_image, validation_image):
        input_image = self.embeder(input_image)
        validation_image = self.embeder(validation_image)
        result = torch.abs(input_image - validation_image)
        x = self.classifier(result)
        x = self.sigmoid(x)
        return x
        

In [17]:
model = SiameseNetwork()

# Specify the input size
input_size = (32, 3, 100, 100)

# Move the model to the desired device (e.g., CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

x = torch.randn(input_size).to(device)
y = torch.randn(input_size).to(device)
test = model(x,y)


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Create an instance of the SiameseNetwork
siamese_net = SiameseNetwork().to(device)

# Define the loss function (e.g., Binary Cross Entropy)
criterion =  nn.BCELoss()

# Define the optimizer (e.g., Stochastic Gradient Descent)
optimizer = optim.Adam(siamese_net.parameters(), lr=1e-4)

# Training loop
num_epochs = 10


for epoch in range(num_epochs):
    train_loss = 0.0
    total_samples = 0
    # Iterate over the training dataset
    for i, (input_image, validation_image, label) in enumerate(train_loader):     
        optimizer.zero_grad()  # Zero the gradients
        
        input_image = input_image.to(device)
        validation_image = validation_image.to(device)
        label = label.to(device)
        # Forward pass
        output = siamese_net(input_image, validation_image)
        # Compute the loss
        loss = criterion(output, label)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update the running loss
        train_loss += loss.item()
        total_samples += label.size(0)  # Increment total_samples by the batch size
    
    avg_train_loss = train_loss / total_samples
    
    print(f"Epoch [{epoch+1}/{num_epochs}]")

    
        # Evaluate the model on the validation set
    val_loss = 0.0
    val_total = 0
    total_samples = 0
    correct_val = 0
    with torch.no_grad():
        for i, (input_image, validation_image, label) in enumerate(val_loader):
            
            input_image = input_image.to(device)
            validation_image = validation_image.to(device)
            label = label.to(device)
            
            # Move inputs and labels to the GPU
            outputs = siamese_net(input_image, validation_image)
            _, predicted = torch.max(outputs.data, 1)
            correct_val += (predicted == label).sum().item()   
                
            loss = criterion(outputs, label)
            val_loss += loss.item()
            total_samples += label.size(0)  # Increment total_samples by the batch size
        
        avg_val_loss = val_loss / total_samples

        print(f"Val Total Loss %f :" %(val_loss))
        print(f"Val Average Loss %f :" % (avg_val_loss))


Epoch [1/10]
Val Total Loss 3.025530 :
Val Average Loss 0.002573 :
Epoch [2/10]
Val Total Loss 0.194077 :
Val Average Loss 0.000165 :
Epoch [3/10]
Val Total Loss 0.131449 :
Val Average Loss 0.000112 :
Epoch [4/10]
Val Total Loss 0.542245 :
Val Average Loss 0.000461 :
Epoch [5/10]
Val Total Loss 0.040660 :
Val Average Loss 0.000035 :
Epoch [6/10]
Val Total Loss 0.040771 :
Val Average Loss 0.000035 :
Epoch [7/10]
Val Total Loss 0.007611 :
Val Average Loss 0.000006 :
Epoch [8/10]
Val Total Loss 0.036195 :
Val Average Loss 0.000031 :
Epoch [9/10]
Val Total Loss 0.031115 :
Val Average Loss 0.000026 :
Epoch [10/10]
Val Total Loss 0.021470 :
Val Average Loss 0.000018 :


# Testing

In [38]:
TEST_NEG_PATH = os.path.join("..","test","NEG")
TEST_POS_PATH = os.path.join("..","test","POS")
TEST_ANC_PATH = os.path.join("..","test","ANC")

transform = transforms.Compose([transforms.Resize((100,100)),
                                transforms.ToTensor()])

postive = MergeImageDataset(TEST_ANC_PATH, TEST_POS_PATH, TEST_NEG_PATH, types = 1, transform = transform)
negtive = MergeImageDataset(TEST_ANC_PATH, TEST_POS_PATH, TEST_NEG_PATH, types = 0, transform = transform)
data = ConcatDataset([negtive,positive])

In [39]:
test_loader_pos = DataLoader(postive)
test_loader_neg = DataLoader(negtive)

In [41]:
with torch.no_grad():
    for i, (input_image, validation_image, label) in enumerate(test_loader_pos):
            input_image = input_image.to(device)
            validation_image = validation_image.to(device)
            output = siamese_net(input_image, validation_image)
            result = 1 if output >= 0.5 else 0
            print(f"predicted {result} and true is {label} with this output{output}")
            print('-------------------------------------------')


    for i, (input_image, validation_image, label) in enumerate(test_loader_neg):
            input_image = input_image.to(device)
            validation_image = validation_image.to(device)
            output = siamese_net(input_image, validation_image)
            result = 1 if output >= 0.5 else 0
            print(f"predicted {result} and true is {label} with this output{output}")
            print('-------------------------------------------')

predicted 0 and true is tensor([[1.]]) with this outputtensor([[0.4994]], device='cuda:0')
-------------------------------------------
predicted 0 and true is tensor([[0.]]) with this outputtensor([[8.4042e-14]], device='cuda:0')
-------------------------------------------


In [27]:
def imgshow(image):
    image = np.squeeze(image) 
    image = np.transpose(image, (1, 2, 0))
    plt.figure(figsize=(10, 8)) 
    plt.imshow(image)  # Display the image using a grayscale colormap
    plt.show()